# Cost Monitoring and Reporting

This notebook demonstrates how to monitor and analyze AWS costs for the MLOps SageMaker Demo project. It uses the AWS Cost Explorer API to retrieve cost data and generate reports.

## Features

- Track costs by service, resource, and tag
- Generate cost forecasts
- Create cost alerts and budgets
- Generate comprehensive cost reports
- Visualize cost data with interactive charts

## Prerequisites

- AWS account with Cost Explorer enabled
- AWS CLI configured with "ab" profile
- Appropriate IAM permissions for Cost Explorer API

Let's start by importing the necessary libraries and setting up our environment.

In [ ]:
import os
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from IPython.display import display, HTML
import ipywidgets as widgets
import json

# Import cost tracking module
import sys
sys.path.append('..')
from src.monitoring.cost_tracking import CostTracker, create_cost_alert, create_cost_dashboard

# Set up visualization
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

# Load project configuration
from configs.project_config import PROJECT_NAME, DATA_BUCKET

# Create AWS session with "ab" profile
session = boto3.Session(profile_name='ab')
account_id = session.client('sts').get_caller_identity()['Account']
region = session.region_name

print(f"Project: {PROJECT_NAME}")
print(f"AWS Account ID: {account_id}")
print(f"Region: {region}")

## 1. Initialize Cost Tracker

Let's initialize the `CostTracker` class to track and analyze costs.

In [ ]:
# Initialize cost tracker
cost_tracker = CostTracker(session=session)

# Create date range widgets
start_date = widgets.DatePicker(
    description='Start Date:',
    value=(datetime.now() - timedelta(days=30)).date(),
    style={'description_width': 'initial'}
)

end_date = widgets.DatePicker(
    description='End Date:',
    value=datetime.now().date(),
    style={'description_width': 'initial'}
)

granularity = widgets.Dropdown(
    options=['DAILY', 'MONTHLY'],
    value='DAILY',
    description='Granularity:',
    style={'description_width': 'initial'}
)

display(start_date, end_date, granularity)

## 2. Get Project Costs

Let's retrieve and analyze the costs for the project.

In [ ]:
# Create function to get project costs
def get_project_costs():
    start = start_date.value.strftime('%Y-%m-%d')
    end = end_date.value.strftime('%Y-%m-%d')
    gran = granularity.value
    
    print(f"Getting project costs from {start} to {end} with {gran} granularity...")
    
    response = cost_tracker.get_project_costs(start, end, gran)
    
    # Calculate total cost
    total_cost = 0
    for result in response.get('ResultsByTime', []):
        for group in result.get('Groups', []):
            total_cost += float(group['Metrics']['UnblendedCost']['Amount'])
    
    print(f"Total cost: ${total_cost:.2f}")
    
    # Display cost data
    cost_data = []
    for result in response.get('ResultsByTime', []):
        time_period = result['TimePeriod']
        start_period = time_period['Start']
        end_period = time_period['End']
        
        for group in result.get('Groups', []):
            service = group['Keys'][0]
            amount = float(group['Metrics']['UnblendedCost']['Amount'])
            unit = group['Metrics']['UnblendedCost']['Unit']
            
            cost_data.append({
                'Start Date': start_period,
                'End Date': end_period,
                'Service': service,
                'Amount': amount,
                'Unit': unit
            })
    
    if cost_data:
        df = pd.DataFrame(cost_data)
        display(df)
        return df
    else:
        print("No cost data available")
        return None

# Create button to get costs
get_costs_button = widgets.Button(
    description='Get Project Costs',
    button_style='info',
    tooltip='Click to get project costs'
)

costs_output = widgets.Output()

def on_get_costs_clicked(b):
    with costs_output:
        costs_output.clear_output()
        df = get_project_costs()

get_costs_button.on_click(on_get_costs_clicked)

display(get_costs_button, costs_output)

## 3. Visualize Costs by Service

Let's create visualizations to analyze costs by service.

In [ ]:
# Create function to plot costs by service
def plot_costs_by_service():
    start = start_date.value.strftime('%Y-%m-%d')
    end = end_date.value.strftime('%Y-%m-%d')
    gran = granularity.value
    
    print(f"Plotting costs by service from {start} to {end} with {gran} granularity...")
    
    fig = cost_tracker.plot_costs_by_service(start, end, gran, top_n=5, figsize=(12, 6))
    plt.show()
    
    # Plot cost breakdown
    print("\nCost breakdown by service:")
    fig = cost_tracker.plot_cost_breakdown(start, end, figsize=(15, 6))
    plt.show()

# Create button to plot costs
plot_costs_button = widgets.Button(
    description='Plot Costs by Service',
    button_style='success',
    tooltip='Click to plot costs by service'
)

plot_output = widgets.Output()

def on_plot_costs_clicked(b):
    with plot_output:
        plot_output.clear_output()
        plot_costs_by_service()

plot_costs_button.on_click(on_plot_costs_clicked)

display(plot_costs_button, plot_output)

## 4. Get SageMaker Costs

Let's analyze the costs specifically for SageMaker resources.

In [ ]:
# Create function to get SageMaker costs
def get_sagemaker_costs():
    start = start_date.value.strftime('%Y-%m-%d')
    end = end_date.value.strftime('%Y-%m-%d')
    gran = granularity.value
    
    print(f"Getting SageMaker costs from {start} to {end} with {gran} granularity...")
    
    response = cost_tracker.get_sagemaker_costs_by_resource(start, end, gran)
    
    # Calculate total SageMaker cost
    total_cost = 0
    for result in response.get('ResultsByTime', []):
        for group in result.get('Groups', []):
            total_cost += float(group['Metrics']['UnblendedCost']['Amount'])
    
    print(f"Total SageMaker cost: ${total_cost:.2f}")
    
    # Display cost data
    cost_data = []
    for result in response.get('ResultsByTime', []):
        time_period = result['TimePeriod']
        start_period = time_period['Start']
        end_period = time_period['End']
        
        for group in result.get('Groups', []):
            resource = group['Keys'][0]
            amount = float(group['Metrics']['UnblendedCost']['Amount'])
            unit = group['Metrics']['UnblendedCost']['Unit']
            
            cost_data.append({
                'Start Date': start_period,
                'End Date': end_period,
                'Resource': resource,
                'Amount': amount,
                'Unit': unit
            })
    
    if cost_data:
        df = pd.DataFrame(cost_data)
        display(df)
        
        # Create pie chart of SageMaker costs by resource
        resource_costs = df.groupby('Resource')['Amount'].sum().sort_values(ascending=False)
        
        plt.figure(figsize=(10, 6))
        plt.pie(
            resource_costs,
            labels=resource_costs.index,
            autopct='%1.1f%%',
            startangle=90,
            shadow=False
        )
        plt.axis('equal')
        plt.title('SageMaker Costs by Resource')
        plt.show()
        
        return df
    else:
        print("No SageMaker cost data available")
        return None

# Create button to get SageMaker costs
get_sagemaker_costs_button = widgets.Button(
    description='Get SageMaker Costs',
    button_style='info',
    tooltip='Click to get SageMaker costs'
)

sagemaker_costs_output = widgets.Output()

def on_get_sagemaker_costs_clicked(b):
    with sagemaker_costs_output:
        sagemaker_costs_output.clear_output()
        df = get_sagemaker_costs()

get_sagemaker_costs_button.on_click(on_get_sagemaker_costs_clicked)

display(get_sagemaker_costs_button, sagemaker_costs_output)

## 5. Get Cost Forecast

Let's get a forecast of future costs.

In [ ]:
# Create function to get cost forecast
def get_cost_forecast():
    start = datetime.now().strftime('%Y-%m-%d')
    end = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')
    
    print(f"Getting cost forecast from {start} to {end}...")
    
    response = cost_tracker.get_cost_forecast(start, end, 'MONTHLY', 80)
    
    # Calculate total forecast
    total_forecast = 0
    for result in response.get('ForecastResultsByTime', []):
        total_forecast += float(result['MeanValue'])
    
    print(f"Total forecast: ${total_forecast:.2f}")
    
    # Display forecast data
    forecast_data = []
    for result in response.get('ForecastResultsByTime', []):
        time_period = result['TimePeriod']
        start_period = time_period['Start']
        end_period = time_period['End']
        mean_value = float(result['MeanValue'])
        
        forecast_data.append({
            'Start Date': start_period,
            'End Date': end_period,
            'Mean Forecast': mean_value,
            'Unit': 'USD'
        })
    
    if forecast_data:
        df = pd.DataFrame(forecast_data)
        display(df)
        
        # Create bar chart of forecast
        plt.figure(figsize=(10, 6))
        plt.bar(
            range(len(df)),
            df['Mean Forecast'],
            tick_label=[f"{row['Start Date']} to {row['End Date']}" for _, row in df.iterrows()]
        )
        plt.title('Cost Forecast')
        plt.ylabel('Forecast (USD)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        return df
    else:
        print("No forecast data available")
        return None

# Create button to get forecast
get_forecast_button = widgets.Button(
    description='Get Cost Forecast',
    button_style='warning',
    tooltip='Click to get cost forecast'
)

forecast_output = widgets.Output()

def on_get_forecast_clicked(b):
    with forecast_output:
        forecast_output.clear_output()
        df = get_cost_forecast()

get_forecast_button.on_click(on_get_forecast_clicked)

display(get_forecast_button, forecast_output)

## 6. Generate Cost Report

Let's generate a comprehensive cost report.

In [ ]:
# Create widgets for report generation
report_format = widgets.Dropdown(
    options=['html', 'markdown', 'json'],
    value='html',
    description='Format:',
    style={'description_width': 'initial'}
)

report_filename = widgets.Text(
    value=f"cost_report_{datetime.now().strftime('%Y%m%d')}",
    description='Filename:',
    style={'description_width': 'initial'}
)

display(report_format, report_filename)

In [ ]:
# Create function to generate cost report
def generate_cost_report():
    start = start_date.value.strftime('%Y-%m-%d')
    end = end_date.value.strftime('%Y-%m-%d')
    format_type = report_format.value
    filename = report_filename.value
    
    if not filename.endswith(f'.{format_type}'):
        filename = f"{filename}.{format_type}"
    
    print(f"Generating {format_type} cost report from {start} to {end}...")
    
    report = cost_tracker.generate_cost_report(
        start_date=start,
        end_date=end,
        output_format=format_type,
        output_file=filename
    )
    
    print(f"Report saved to: {filename}")
    
    # Display report preview
    if format_type == 'html':
        display(HTML(report))
    elif format_type == 'markdown':
        print(report[:1000] + '...' if len(report) > 1000 else report)
    else:  # json
        print(report[:1000] + '...' if len(report) > 1000 else report)

# Create button to generate report
generate_report_button = widgets.Button(
    description='Generate Report',
    button_style='danger',
    tooltip='Click to generate cost report'
)

report_output = widgets.Output()

def on_generate_report_clicked(b):
    with report_output:
        report_output.clear_output()
        generate_cost_report()

generate_report_button.on_click(on_generate_report_clicked)

display(generate_report_button, report_output)

## 7. Create Cost Budget

Let's create a cost budget to help control costs.

In [ ]:
# Create widgets for budget creation
budget_name = widgets.Text(
    value=f"{PROJECT_NAME}-budget-{datetime.now().strftime('%Y%m')}",
    description='Budget Name:',
    style={'description_width': 'initial'}
)

budget_amount = widgets.FloatText(
    value=100.0,
    description='Amount (USD):',
    style={'description_width': 'initial'}
)

notification_email = widgets.Text(
    value="",
    description='Email:',
    placeholder='Enter notification email',
    style={'description_width': 'initial'}
)

threshold_percent = widgets.FloatSlider(
    value=80.0,
    min=50.0,
    max=100.0,
    step=5.0,
    description='Threshold (%):',
    style={'description_width': 'initial'}
)

display(budget_name, budget_amount, notification_email, threshold_percent)

In [ ]:
# Create function to create budget
def create_budget():
    name = budget_name.value
    amount = budget_amount.value
    email = notification_email.value
    threshold = threshold_percent.value
    
    print(f"Creating budget '{name}' with amount ${amount:.2f}...")
    
    response = cost_tracker.create_cost_budget(
        budget_name=name,
        budget_amount=amount,
        notification_email=email if email else None,
        threshold_percent=threshold
    )
    
    if response:
        print(f"Budget created successfully")
        if email:
            print(f"Notification will be sent to {email} when threshold of {threshold}% is reached")
    else:
        print("Failed to create budget")

# Create button to create budget
create_budget_button = widgets.Button(
    description='Create Budget',
    button_style='primary',
    tooltip='Click to create cost budget'
)

budget_output = widgets.Output()

def on_create_budget_clicked(b):
    with budget_output:
        budget_output.clear_output()
        create_budget()

create_budget_button.on_click(on_create_budget_clicked)

display(create_budget_button, budget_output)

## 8. Create Cost Dashboard

Let's create a CloudWatch dashboard for cost monitoring.

In [ ]:
# Create widget for dashboard name
dashboard_name = widgets.Text(
    value=f"{PROJECT_NAME}-cost-dashboard",
    description='Dashboard Name:',
    style={'description_width': 'initial'}
)

display(dashboard_name)

In [ ]:
# Create function to create dashboard
def create_dashboard():
    name = dashboard_name.value
    
    print(f"Creating dashboard '{name}'...")
    
    response = create_cost_dashboard(
        dashboard_name=name,
        session=session
    )
    
    if response:
        print(f"Dashboard created successfully")
        print(f"Dashboard URL: https://{session.region_name}.console.aws.amazon.com/cloudwatch/home?region={session.region_name}#dashboards:name={name}")
    else:
        print("Failed to create dashboard")

# Create button to create dashboard
create_dashboard_button = widgets.Button(
    description='Create Dashboard',
    button_style='success',
    tooltip='Click to create cost dashboard'
)

dashboard_output = widgets.Output()

def on_create_dashboard_clicked(b):
    with dashboard_output:
        dashboard_output.clear_output()
        create_dashboard()

create_dashboard_button.on_click(on_create_dashboard_clicked)

display(create_dashboard_button, dashboard_output)

## 9. Cost Optimization Recommendations

Let's generate cost optimization recommendations based on the cost data.

In [ ]:
# Create function to generate recommendations
def generate_recommendations():
    start = start_date.value.strftime('%Y-%m-%d')
    end = end_date.value.strftime('%Y-%m-%d')
    
    print(f"Generating cost optimization recommendations based on data from {start} to {end}...")
    
    # Get cost data
    project_costs = cost_tracker.get_project_costs(start, end, 'MONTHLY')
    sagemaker_costs = cost_tracker.get_sagemaker_costs_by_resource(start, end, 'MONTHLY')
    
    # Calculate total cost
    total_cost = 0
    for result in project_costs.get('ResultsByTime', []):
        for group in result.get('Groups', []):
            total_cost += float(group['Metrics']['UnblendedCost']['Amount'])
    
    # Calculate SageMaker cost
    sagemaker_cost = 0
    for result in sagemaker_costs.get('ResultsByTime', []):
        for group in result.get('Groups', []):
            sagemaker_cost += float(group['Metrics']['UnblendedCost']['Amount'])
    
    # Generate recommendations
    recommendations = []
    
    # SageMaker recommendations
    if sagemaker_cost > 0:
        sagemaker_percentage = (sagemaker_cost / total_cost * 100) if total_cost > 0 else 0
        
        if sagemaker_percentage > 50:
            recommendations.append({
                'category': 'SageMaker',
                'title': 'High SageMaker Costs',
                'description': f'SageMaker accounts for {sagemaker_percentage:.1f}% of your total costs.',
                'recommendations': [
                    'Use spot instances for training jobs to reduce costs by up to 90%',
                    'Implement auto-scaling for endpoints to match demand',
                    'Use smaller instance types for development and testing',
                    'Terminate unused endpoints and notebook instances'
                ]
            })
        
        # Check for training job costs
        training_cost = 0
        for result in sagemaker_costs.get('ResultsByTime', []):
            for group in result.get('Groups', []):
                resource = group['Keys'][0]
                if 'training' in resource.lower():
                    training_cost += float(group['Metrics']['UnblendedCost']['Amount'])
        
        if training_cost > 0.3 * sagemaker_cost:
            recommendations.append({
                'category': 'Training',
                'title': 'High Training Costs',
                'description': f'Training jobs account for {(training_cost / sagemaker_cost * 100):.1f}% of your SageMaker costs.',
                'recommendations': [
                    'Use spot instances for training jobs',
                    'Optimize hyperparameter tuning jobs to use fewer trials',
                    'Use smaller instance types for development and testing',
                    'Implement early stopping to terminate underperforming jobs'
                ]
            })
    
    # General recommendations
    recommendations.append({
        'category': 'General',
        'title': 'Cost Optimization Best Practices',
        'description': 'General cost optimization recommendations.',
        'recommendations': [
            'Implement resource tagging to track costs by feature or team',
            'Set up cost budgets and alerts to monitor spending',
            'Schedule automatic shutdown of development resources when not in use',
            'Regularly review and clean up unused resources',
            'Use the cleanup scripts provided in the demo to terminate resources when done'
        ]
    })
    
    # Display recommendations
    for rec in recommendations:
        print(f"\n## {rec['title']}")
        print(f"{rec['description']}\n")
        print("Recommendations:")
        for item in rec['recommendations']:
            print(f"- {item}")

# Create button to generate recommendations
generate_recommendations_button = widgets.Button(
    description='Generate Recommendations',
    button_style='info',
    tooltip='Click to generate cost optimization recommendations'
)

recommendations_output = widgets.Output()

def on_generate_recommendations_clicked(b):
    with recommendations_output:
        recommendations_output.clear_output()
        generate_recommendations()

generate_recommendations_button.on_click(on_generate_recommendations_clicked)

display(generate_recommendations_button, recommendations_output)

## Conclusion

In this notebook, we've demonstrated how to monitor and analyze AWS costs for the MLOps SageMaker Demo project. We've shown how to:

1. Track costs by service, resource, and tag
2. Generate cost forecasts
3. Create cost alerts and budgets
4. Generate comprehensive cost reports
5. Visualize cost data with interactive charts
6. Generate cost optimization recommendations

These tools can help you monitor and control costs for your MLOps projects on AWS.